In [ ]:
%load_ext autoreload
%autoreload 2

from edunn import utils
import numpy as np

# Capa AddConstant

En este ejercicio debés implementar la capa `AddConstant`, que agrega un valor constante a cada una de sus entradas para generar su salida.

Por ejemplo, si la entrada `x` es `[3.5,-7.2,5.3]` y la capa AddConstant agrega el valor `3.0` a su entrada, entonces la salida `y` será `[6.5,-4.2,8.3]`.

Tu objetivo es implementar los métodos `forward` y `backward` de esta capa, de modo de poder utilizarla en una red neuronal.

Esta capa funciona para arreglos de entrada de cualquier tamaño, ya sean vectores, matrices, o arreglos con más dimensiones.


# Método forward

El método `forward` calcula la salida `y` en base a la entrada `x`, como explicamos antes. En términos formales, si la constante a sumar es $C$ y la entrada a la capa es $x = [x_1,x_2,...,x_n]$, entonces la salida $y$ es:

$
y([x_1,x_2,...,x_n])= [x_1+C,x_2+C,...,x_n+C]
$

Comenzamos con el método `forward` de la clase `AddConstant`. Debés completar el código entre los comentarios:

```""" YOUR IMPLEMENTATION START """```

y

```""" YOUR IMPLEMENTATION END """```

Entre esas líneas hay código por defecto (en este caso `y = np.zeros_like(x)`) solo para evitar errores del intérprete de Python hasta que escribas tu implementación. Podés borrar esa línea para tener una implementación limpia! 

No olvides ejecutar la celda después de agregar tu código!

Luego, verificar con la siguiente celda una capa que suma 3 y otra que suma -3. Si ambos chequeos son correctos, verás dos mensajes de <span style='background-color:green;color:white; '>éxito (success)</span>.

In [ ]:
from edunn import Model

class AddConstant(Model):
    """
    A layer that adds a constant. This layer has NO parameters
    """

    def __init__(self, value: float, name=None):
        super().__init__(name=name)
        self.value = value

    def forward(self, x: np.ndarray):
        """
        :param x: input vector/matrix
        :return: `x + a`, constant value, stored in `self.value`
        """

        """ YOUR IMPLEMENTATION START """
        y = np.zeros_like(x)
        """ YOUR IMPLEMENTATION END """

        return y

    def backward(self, dE_dy: np.ndarray):
        """ YOUR IMPLEMENTATION START """
        dE_dx = np.zeros_like(dE_dy)
        """ YOUR IMPLEMENTATION END """

        dE_dp = {}  # no parameters, no derivatives
        return dE_dx, dE_dp


In [ ]:
x = np.array([[3.5, -7.2, 5.3],
              [-3.5, 7.2, -5.3]])

layer = AddConstant(3)
y = np.array([[6.5, -4.2, 8.3],
              [-0.5, 10.2, -2.3]])
utils.check_same(y, layer.forward(x))

layer = AddConstant(-3)
y = np.array([[0.5, -10.2, 2.3],
              [-6.5, 4.2, -8.3]])
utils.check_same(y, layer.forward(x))

# Método backward

Además del cálculo de la salida de la capa, la misma debe poder propagar hacia atrás el gradiente del error de la red. Para eso, debés implementar el método `backward` que recibe $\frac{dE}{dy}$, es decir, las derivadas parciales del error respecto a la salida (gradiente) de esta capa , y devolver $\frac{dE}{dx}$, las derivadas parciales del error respecto de las entradas de esta capa. 

Para la capa `AddConstant` el cálculo del gradiente de la salida $y$ respecto de la entrada $x$ es fácil, pero recordemos que tenemos que devolver el gradiente de $E$ respecto de $x$. Recordemos entonces la forma de la salida:

$
y([x_1,x_2,...,x_n])= [x_1+C,x_2+C,...,x_n+C]
$

Nuestro objetivo es calcular $\frac{dE}{dx}$. Para eso, nos enfocaremos en $\frac{dE}{dx_i}$, es decir la derivada del error respecto de una entrada en particular. 
Luego, aplicando regla de la cadena, podemos escribir $\frac{dE}{dx_i}$ como:


$\frac{dE}{dx_i} = \frac{dE}{dy} * \frac{dy}{dx_i} = \sum_j \frac{dE}{dy_j} * \frac{dy_j}{dx_i} $


Nos conviene entonces ver cada elemento de la salida escrito como:

$y_i(x)=x_i+C$

Y por ende:

$\frac{dy_i}{dx_i} = 1 + 0  =1$

Como no hay interacción entre elementos de distinto índice, es decir, $y_i$ solo depende de $x_i$, en la sumatoria anterior si $i\neq j$  entonces $\frac{dy_j}{dx_i}=0$. Por eso podemos quitar la sumatoria y ahora utilizar la regla de la cadena solo con $y_i$:

$\frac{dE}{dx_i} = \frac{dE}{dy} * \frac{dy}{dx_i} = \sum_j \frac{dE}{dy_j} * \frac{dy_j}{dx_i} =  \frac{dE}{dy_i} * \frac{dy_i}{dx_i} $

Sabiendo que  $\frac{dy_i}{dx_i} = 1$

$ \frac{dE}{dx_i} = \frac{dE}{dy} * \frac{dy}{dx_i} = \frac{dE}{dy_i} * 1 = \frac{dE}{dy_i} $

Escribiendo entonces en forma vectorial para el vector x:

$ \frac{dE}{dx} = [ \frac{dE}{dy_1}, \frac{dE}{dy_2}, ..., \frac{dE}{dy_n} ] = \frac{dE}{dy} $

Con lo cual la capa simplemente propaga los gradientes de la capa siguiente.

Notar que por simplicidad, en el código llamamos a estos vectores `dEdy` y `dEdx`. También aclaramos que en este caso $C$ es una constante y NO un parámetro de la red, por lo cual no debemos calcular $\frac{dE}{dC}$.

La verificación del gradiente se hace con la función `check_gradient_layer_random_sample`. Esta función genera muestras aleatorias de `x` y de `dEdy`, y luego compara el gradiente analítico (tu implementación) contra el gradiente numérico. 

El gradiente numérico _aproxima_ las derivadas parciales utilizando la fórmula de la derivada $\frac{df(x)}{dx}= \lim_{h→0} \frac{f(x+h)-f(x)}{h}$ con un valor de $h$ muy pequeño ($h=10^{-12}$). En realidad, para una una mejor aproximación, utiliza la derivada _centrada_, cuya fórmula es $\frac{df(x)}{dx}_{aprox}= \frac{f(x+h)-f(x-h)}{2h}$. Esta técnica de _verificación de gradientes_ es una técnica estándar para comprobar la implementación correcta de una red neuronal.

Completar el código en la función `backward` de la capa `AddConstant` entre los comentarios:

```""" YOUR IMPLEMENTATION START """```

y

```""" YOUR IMPLEMENTATION END """```

No olvides ejecutar la celda después de agregar tu código!

Y luego verificar con la siguiente celda una capa que suma 3 y otra que suma -3. Si ambos chequeos son correctos, verás dos mensajes de <span style='background-color:green;color:white; '>éxito (success)</span>.

In [ ]:
from edunn.utils import check_gradient

# number of random values of x and dEdy to generate and test gradients
samples = 100

input_shape = (5, 2)

# Test derivatives of an AddConstant layer that adds 3
layer = AddConstant(3)
check_gradient.common_layer(layer, input_shape, samples=samples)

# Test derivatives of an AddConstant layer that adds -4
layer = AddConstant(-4)
check_gradient.common_layer(layer, input_shape, samples=samples)


# Nombre de capa

Los nombres de la capa se asignan de forma automática al crearse un objeto de la misma, e idealmente deben ser únicos para poder diferenciar distintas capas aunque sean del mismo tipo.

Por defecto, al ejecutar `AddConstant(3)` se crea un objeto de esta capa, y se le pone el nombre de la `AddConstant_i` donde `i` va incrementándose automáticamente a medida que creamos objetos de la misma clase `AddConstant`. 

También se puede especificar el nombre de la capa manualmente, para que quede fijo, utilizando el parámetro `name`, por ejemplo con `AddConstant(3, name="Una capa que suma 3")`

Todas las capas deben seguir la convención de tener un parámetro `name` para que la librería las identifique.

In [ ]:
c1 = AddConstant(3)
print(c1.name)

c2 = AddConstant(3)
print(c2.name)

c3 = AddConstant(3, name="Mi primera capa :)")
print(c3.name)
